# Checkpointing in Workflows

During the execution of a Workflow, `Checkpoint` objects are automatically stored within the `Context` class. By default, `Checkpoints` are automatically created at the completion of a step. At such instants in the Workflow execution, we checkpoint the name of the last completed step, it's input `Event`, it's output `Event`, as well as a snapshot of the `Context` itself. We also store a `Checkpoint` before the emission of the `StartEvent`, but note that running from these special checkpoints are essentially equivalent to the typical execution of a `Workflow` -- we add it here purely for convenience.

In this notebook, we demonstrate how to make use of such checkpoints. Specifically, we'll show how to obtain the list of all of the stored checkpoints, filter this list, and even how to re-run a `Workflow` from any one them.

## Defining the Workflow

In [ ]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core.workflow import (
    Workflow,
    step,
    StartEvent,
    StopEvent,
    Event,
    Context,
)
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    llm = OpenAI(api_key=api_key)

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))

### Running the Workflow

In [ ]:
# instantiate Jokeflow
workflow = JokeFlow()
ctx = Context(workflow=workflow)

# run it
handler = workflow.run(ctx=ctx, topic="math")
await handler

'Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from personifying the equal sign and attributing human emotions and characteristics to it. The punchline relies on a play on words, as "less than" and "greater than" are mathematical symbols used to show inequality.\n\nCritique:\n- Clever wordplay: The joke cleverly uses mathematical symbols and concepts to create humor. The play on words with "less than" and "greater than" adds an extra layer of wit to the joke.\n- Relatable: The joke is relatable to anyone familiar with basic math, making it accessible to a wide audience.\n- Simple and straightforward: The joke is easy to understand and doesn\'t require any complex setup or explanation, making it effective for quick, lighthearted humor.\n- Lack of depth: While the joke is amusing, it may be considered somewhat shallow or predictable. The punchline is fairly straightforward and doesn\'t off

### Listing the saved Checkpoints

In [ ]:
handler.ctx.checkpoints

[Checkpoint(id_='0edf950e-2805-4ade-a67f-55d0d1225e4c', last_completed_step=None, input_event=None, output_event=StartEvent(), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'checkpoints': [], 'is_running': False}),
 Checkpoint(id_='4f7f257f-36c7-4cbc-9339-9622393f2584', last_completed_step='generate_joke', input_event=StartEvent(), output_event=JokeEvent(joke="Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else."), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'checkpoints': ['{"__is_pydantic": true, "value": {"id_": "0edf950e-2805-4ade-

In [ ]:
len(handler.ctx.checkpoints)

3

There are 3 stored checkpoints after this first run. The first of these checkpoints are created just before the emission of the `StartEvent`. This can be thought of as a "startup" step being completed which has a null input `Event` and whose output event is the `StartEvent`. The remaining two of these checkpoints are made at the completion of each of the two steps in this Workflow. 

### Filtering the Checkpoints

Before showcasing how to filter through checkpoints, let's first make things a bit more interesting by executing the workflow a few more times. Here, we'll run the workflow for two more additional topics. Since each entire run will result in 3 stored checkpoints each, we should have a total of 9 checkpoints after executing the below cell. 

In [ ]:
additional_topics = ["biology", "history"]

for topic in additional_topics:
    handler = workflow.run(
        ctx=ctx, topic=topic
    )  # same Context that was used before
    await handler

len(handler.ctx.checkpoints)

9

In [ ]:
# Filter by output event StopEvent
checkpoints_that_emit_stop_event = ctx.filter_checkpoints(
    output_event_type=StopEvent
)
len(checkpoints_that_emit_stop_event)
[ckpt.output_event.result[:150] for ckpt in checkpoints_that_emit_stop_event]

['Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from ',
 'This joke plays on the idea of biologists studying genes and mathematicians working with numbers, creating a humorous scenario where the two disciplin',
 'Analysis:\nThis joke plays on the double meaning of the word "pupils," which can refer to both the students in a classroom and the black part of the ey']

In [ ]:
# Filter by the name of last completed step
checkpoints_right_after_generate_joke_step = ctx.filter_checkpoints(
    last_completed_step="generate_joke"
)
len(checkpoints_right_after_generate_joke_step)
[
    ckpt.output_event.joke[:150]
    for ckpt in checkpoints_right_after_generate_joke_step
]

["Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else.",
 "Why did the biologist break up with the mathematician?\n\nBecause they couldn't find a common gene!",
 "Why did the history teacher go to jail?\n\nBecause he couldn't control his pupils!"]

### Re-Run Workflow from a specific checkpoint

In [ ]:
ckpt = checkpoints_right_after_generate_joke_step[1]
handler = workflow.run_from(checkpoint=ckpt, ctx=ctx)
await handler

'This joke plays on the idea of a biologist and a mathematician being in a relationship, which is already a humorous concept due to the perceived differences in their fields of study. The punchline, "they couldn\'t find a common gene," is a clever play on words, as "gene" can refer to a unit of heredity in biology, but in this context, it is used to mean a common interest or trait.\n\nOverall, the joke is light-hearted and plays on the stereotypes of biologists and mathematicians having different ways of thinking and approaching problems. It also cleverly combines scientific terminology with a common relationship issue, making it accessible to a wide audience.\n\nHowever, some may find the joke to be a bit simplistic or predictable, as the punchline is somewhat telegraphed by the setup. Additionally, the joke relies heavily on stereotypes about scientists, which may not resonate with everyone. \n\nIn conclusion, while the joke is clever and plays on the differences between biologists a

Since we've executed from the checkpoint that represents the end of "generate_joke" step, there is only one additional checkpoint that gets stored i.e., the one that is stored after the completion of the next step, namely "critique_joke".

In [ ]:
len(ctx.checkpoints)

10

### Disabling Automatic Checkpointing

By default, automatic checkpointing is enabled. However, this can be disabled for any run via the `store_checkpoints` parameter.

In [ ]:
handler = workflow.run(topic="chemistry", ctx=ctx, store_checkpoints=False)
await handler

'This joke plays on the double meaning of the word "nitrates." In chemistry, nitrates are a type of compound, while in everyday language, "day rates" refers to the cost of something for a day. The humor comes from the unexpected comparison between the two meanings.\n\nOne positive aspect of this joke is its clever wordplay and the unexpected twist at the end. The punchline is unexpected and plays on the audience\'s knowledge of chemistry terms, making it a clever and witty joke for those who understand the reference.\n\nHowever, one potential critique of this joke is that it may not be easily understood by those who are not familiar with chemistry terms. The humor relies heavily on the audience\'s knowledge of nitrates and their cost, which may limit the joke\'s appeal to a more niche audience.\n\nOverall, this joke is a clever play on words that can be appreciated by those with a background in chemistry. It effectively combines science terminology with everyday language to create a hu

In [ ]:
len(ctx.checkpoints)

10

As we can see the number of stored checkpoints in our `Context` has not increased from the run prior to the one just executed above. In other words, the run above didn't produce any new checkpoints since we disabled checkpointing for that run.